In [1]:
from ner import NER

d:\programs\python\python36\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
parameters = {'pretrained_model_folder':'../model', 
                      'dataset_text_folder':'../../../ML_EntityData/data/en', 
                      'character_embedding_dimension':25, 
                      'character_lstm_hidden_state_dimension':25, 
                      'check_for_digits_replaced_with_zeros':True, 
                      'check_for_lowercase':True, 
                      'debug':False,
                      'dropout_rate':0.5,
                      'experiment_name':'test',
                      'freeze_token_embeddings':False,
                      'gradient_clipping_value':5.0,
                      'learning_rate':0.005,
                      'load_only_pretrained_token_embeddings':False,
                      'load_all_pretrained_token_embeddings':False,
#                       'main_evaluation_mode':'conll',
                      'maximum_number_of_epochs':500,
                      'number_of_cpu_threads':8,
                      'number_of_gpus':0,
                      'optimizer':'sgd',
                      'output_folder':'../../../ML_EntityData/output',
                      'patience':10,
#                       'plot_format':'pdf',
                      'reload_character_embeddings':True,
                      'reload_character_lstm':True,
                      'reload_crf':True,
                      'reload_feedforward':True,
                      'reload_token_embeddings':True,
                      'reload_token_lstm':True,
                      'remap_unknown_tokens_to_unk':True,
#                       'spacylanguage':'en',
                      'tagging_format':'bioes',
                      'token_embedding_dimension':100,
                      'token_lstm_hidden_state_dimension':100,
                      'token_pretrained_embedding_filepath':'../../../ML_EntityData/embedding/en/glove.6B.100d.txt',
#                       'tokenizer':'spacy',
                      'train_model':True,
                      'use_character_lstm':True,
#                       'use_crf':True,
                      'use_pretrained_model':False,
                      'verbose':False}

In [3]:
import utils_nlp
import pickle
# dataset_filepaths, dataset_brat_folders = utils.get_valid_dataset_filepaths(parameters)
# dataset = ds.DatasetP(verbose=False, debug=False)
token_to_vector = utils_nlp.load_pretrained_token_embeddings(parameters['token_pretrained_embedding_filepath'])
dataset = pickle.load(open('../model/en/dataset.pickle', 'rb'))

In [4]:
ner = NER(parameters,dataset,token_to_vector)

embedded_characters: Tensor("character_embedding/embedded_characters:0", shape=(?, ?, 25), dtype=float32)
embedded_tokens: Tensor("token_embedding/embedding_lookup:0", shape=(?, 100), dtype=float32)
token_lstm_input: Tensor("concatenate_token_and_character_vectors/token_lstm_input:0", shape=(?, 163), dtype=float32)
token_lstm_input_drop: Tensor("dropout/token_lstm_input_drop/mul:0", shape=(?, 163), dtype=float32)
token_lstm_input_drop_expanded: Tensor("dropout/token_lstm_input_drop_expanded:0", shape=(1, ?, 163), dtype=float32)
unary_scores_expanded: Tensor("crf/unary_scores_expanded:0", shape=(1, ?, 19), dtype=float32)
input_label_indices_flat_batch: Tensor("crf/input_label_indices_flat_batch:0", shape=(1, ?), dtype=int32)
sequence_lengths: Tensor("crf/sequence_lengths:0", shape=(1,), dtype=int32)


d:\programs\python\python36\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Load token embeddings... done (0.18 seconds)
number_of_token_original_case_found: 14618
number_of_token_lowercase_found: 11723
number_of_token_digits_replaced_with_zeros_found: 119
number_of_token_lowercase_and_digits_replaced_with_zeros_found: 16
number_of_loaded_word_vectors: 26476
dataset.vocabulary_size: 28984


In [5]:
ner.restore_model_trained(parameters,model_pathfile=('../model/en/model.ckpt'),
                                        dataset_pathfile=('../model/en/dataset.pickle'),
                                        embedding_filepath= parameters['token_pretrained_embedding_filepath'],
                                        character_dimension = parameters['character_embedding_dimension'],
                                        token_dimension=parameters['token_embedding_dimension'])

INFO:tensorflow:Restoring parameters from ../model/en/model.ckpt


In [6]:
ner.train(parameters,0,'../../../ML_EntityData/output/en/model')


Starting epoch 0
Training completed in 0.00 seconds
Evaluate model on the train set
             precision    recall  f1-score   support

      B-LOC     0.9965    0.9972    0.9968      7140
     B-MISC     0.9985    0.9933    0.9959      3438
      B-ORG     0.9945    0.9970    0.9957      6321
      B-PER     0.9982    0.9967    0.9974      6600
      I-LOC     0.9983    0.9983    0.9983      1157
     I-MISC     0.9965    0.9974    0.9970      1155
      I-ORG     0.9997    0.9968    0.9982      3704
      I-PER     0.9989    0.9996    0.9992      4528
          O     0.0000    0.0000    0.0000        30

avg / total     0.9965    0.9961    0.9963     34073

Evaluate model on the valid set
             precision    recall  f1-score   support

      B-LOC     0.9659    0.9717    0.9688      1837
     B-MISC     0.9283    0.8850    0.9062       922
      B-ORG     0.9313    0.9299    0.9306      1341
      B-PER     0.9620    0.9767    0.9693      1842
      I-LOC     0.9237    0.894

In [7]:
_ , entitys = ner.quick_predict('my name is Jon Jgo Linh.')
entitys

[('Jon Jgo Linh', 'PER')]

In [9]:
import codecs
text_filepath = '../data/test/cv.txt'
with codecs.open(text_filepath, 'r', 'UTF-8') as f:
    text = f.read()
_,entitys = ner.quick_predict(text)
entitys

[('Birth', 'PER'),
 ('Thai Hoc Street', 'LOC'),
 ('Ba Dinh', 'LOC'),
 ('Hanoi', 'LOC'),
 ('iOS Developer', 'ORG'),
 ('iOS', 'LOC'),
 ('watchOS', 'LOC'),
 ('OS X & tvOS', 'ORG'),
 ('Swift', 'MISC'),
 ('iOS', 'MISC'),
 ('ITSOL JSC', 'ORG'),
 ('Emotiv', 'LOC'),
 ('Silicon Valley', 'LOC'),
 ('iOS', 'ORG'),
 ('Metal Commands ”', 'ORG'),
 ('iOS SDK', 'ORG'),
 ('Swift', 'MISC'),
 ('FPT Software', 'ORG'),
 ('Vietnam', 'LOC'),
 ('Core', 'ORG'),
 ('Swift', 'PER'),
 ('iOS', 'MISC'),
 ('Techmaster', 'ORG'),
 ('Swift Course', 'MISC'),
 ('Techmaster Vietnam', 'LOC'),
 ('Vietnam', 'LOC'),
 ('Trinh Minh Cuong', 'PER'),
 ('Swift', 'MISC'),
 ('iOS', 'LOC'),
 ('Projects Freelancer', 'PER'),
 ('US', 'LOC'),
 ('Japanese', 'MISC'),
 ('Skills iOS Development', 'ORG'),
 ('MVVM', 'ORG'),
 ('Pool', 'ORG'),
 ('Optimization', 'LOC'),
 ('Unit Testing', 'ORG'),
 ('CoreData', 'LOC'),
 ('SQLite', 'LOC'),
 ('Realm', 'LOC'),
 ('UIKit', 'ORG'),
 ('AutoLayout', 'LOC'),
 ('Storyboards', 'LOC'),
 ('REST API', 'ORG'),
 ('JS

In [8]:
import sklearn.metrics as metrics


In [9]:
y1=[1,2,3,4,2,3,4]
y2=[2,1,3,3,2,3,4]

In [15]:
[float(t) for t in [t for t in metrics.classification_report(y1,y2).split('\n')[-2].split(' ') if len(t)>0][-4:]]

[0.62, 0.57, 0.56, 7.0]

In [12]:
float('1')

1.0